In [1]:
%load_ext autoreload
%autoreload 2
from models import NNDynamicModel, MPCcontroller
import tensorflow as tf
import numpy as np
import json

/Users/loulinhui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
regularizer = tf.contrib.layers.l2_regularizer(scale=1e-5)
activation = tf.nn.relu
output_activation = None
learning_rate = 0.003
batch_size = 128

In [3]:
# load data
with open('./sample.json', 'r') as f:
    data = json.loads(f.read())
    
print('loaded %d data.' % (len(data)))


loaded 8000 data.


In [4]:
obs = [d[0] for d in data]
normalization = [np.min(obs, axis=0), np.max(obs, axis=0), np.mean(obs, axis=0)]
print(normalization)

session = tf.Session()

[array([  0.   , -50.   ,  90.   ,  17.   ,   6.166]), array([  0.   , 625.   , 105.   ,  51.   ,   6.657]), array([  0.        , 424.27775   ,  97.558125  ,  29.69      ,
         6.21954263])]


In [5]:
model = NNDynamicModel(
    name='test',
    n_layers=3,
    size=128,
    activation=activation,
    output_activation=output_activation,
    kernel_regularizer=regularizer,
    normalization=normalization,
    batch_size=batch_size,
    iterations=40,
    learning_rate=learning_rate,
    sess=session
)

In [6]:
init = tf.global_variables_initializer()
session.run(init)

In [7]:
model.fit(data)

on iter_step 0, loss = 0.1193627
on iter_step 1, loss = 0.0912459
on iter_step 2, loss = 0.0873503
on iter_step 3, loss = 0.0813736
on iter_step 4, loss = 0.0820244
on iter_step 5, loss = 0.0788480
on iter_step 6, loss = 0.0774312
on iter_step 7, loss = 0.0775689
on iter_step 8, loss = 0.0757611
on iter_step 9, loss = 0.0760173
on iter_step 10, loss = 0.0744271
on iter_step 11, loss = 0.0743639
on iter_step 12, loss = 0.0747819
on iter_step 13, loss = 0.0737394
on iter_step 14, loss = 0.0737247
on iter_step 15, loss = 0.0739567
on iter_step 16, loss = 0.0739094
on iter_step 17, loss = 0.0733404
on iter_step 18, loss = 0.0731530
on iter_step 19, loss = 0.0726518
on iter_step 20, loss = 0.0722122
on iter_step 21, loss = 0.0721085
on iter_step 22, loss = 0.0723511
on iter_step 23, loss = 0.0721214
on iter_step 24, loss = 0.0718269
on iter_step 25, loss = 0.0719076
on iter_step 26, loss = 0.0715924
on iter_step 27, loss = 0.0707279
on iter_step 28, loss = 0.0710203
on iter_step 29, loss = 

In [8]:
mpc = MPCcontroller(dyn_model=model) 

In [13]:
nd = [d for d in data if d[-1] < 0]
obs, action, nobs, reward = nd[3]
print(model.predict([obs], [action]))
print(obs, action, nobs, reward)
action = mpc.get_action(obs)
print(action)

(array([[-1.7906092e-03,  3.5971649e+02,  1.0513793e+02,  5.1814770e+01,
         6.2279243e+00]], dtype=float32), array([0.64560527], dtype=float32))
[0.0, 458.0, 105.0, 51.0, 6.204000000000068] [0.0, 0.32596028023061896] [0.0, 333.0, 105.0, 51.0, 6.2260000000000755] -1
[0.         0.93700732]


In [10]:
from game_controller import GameEnv

In [11]:
env = GameEnv()
env.start()

obs = None
step = 1000
while step > 0:
    if env.status == 'CRASHED':
        env.restart()
    if obs is None:
        obs = env.get_observation()
    if obs is not None:
        action = mpc.get_action(obs)
        obs, _, _ = env.perform_action(action)
        step -= 1

TypeError: object of type 'NoneType' has no len()